# 4.2.3.
1. Selecionar aleatoriamente duas amostras de 20 registos dos níveis médios de poluição atmosférica em Espanha e França.
2. Usar estas amostras para testar se os níveis de poluição atmosférica apresentam diferenças significativas entre os dois países.

## *Imports* estáticos

In [59]:
import pandas as pd
from scipy.stats import shapiro, mannwhitneyu, ttest_ind

## Leitura, filtro e seleção aleatória das informações do ficheiro base

In [60]:
dados = pd.read_csv('../dados/AIRPOL_data.csv', delimiter=";", header=0)
dados = dados.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])

num_samples = 20
significancia = 0.05

## Testar normalidade e testar diferença
Aqui usaremos o **teste de Shapiro-Wilk** para ver estes dois *samples* seguem a distribuição normal. Se alguma das *samples* não seguir a distribuição normal (p-value < significancia), usaremos o teste de **Mann-Whitney-U**. Caso contrário (p_value >= significancia), usaremos o teste de **T-Student**.

O objetivo deste teste é verificar se vamos rejeitar ou não a hipótese nula. Para isto, vamos ver se o p_value resultante do teste escolhido é inferior à significância (rejeitamos a hipótese nula) ou não.

Também rodamos o teste com múltiplas seleções para ver qual dos testes e qual resultado é mais provável.

In [61]:
tstudent_vezes = 0
rejeitar_nula_vezes = 0
num_execucoes = 1000
for _ in range(num_execucoes):
    dados_espanha_4_2_3 = dados[dados['Country']=='Spain'].filter(['Air_Pollution_Average[ug/m3]']).sample(n=num_samples)
    dados_franca_4_2_3 = dados[dados['Country']=='France'].filter(['Air_Pollution_Average[ug/m3]']).sample(n=num_samples)
    dados_espanha_4_2_3['Air_Pollution_Average[ug/m3]'] = dados_espanha_4_2_3['Air_Pollution_Average[ug/m3]'].str.replace(',', '.').astype(float)
    dados_franca_4_2_3['Air_Pollution_Average[ug/m3]'] = dados_franca_4_2_3['Air_Pollution_Average[ug/m3]'].str.replace(',', '.').astype(float)

    _, p_value_espanha = shapiro(dados_espanha_4_2_3)
    _, p_value_franca = shapiro(dados_franca_4_2_3)

    if ((p_value_espanha < significancia) | (p_value_franca < significancia)):
        # Teste Mann-Whitney-U
        _, p_value = mannwhitneyu(dados_espanha_4_2_3, dados_franca_4_2_3)
    else:
        # Teste T-Student 
        _, p_value = ttest_ind(dados_espanha_4_2_3, dados_franca_4_2_3)
        tstudent_vezes += 1

    if(p_value < significancia): rejeitar_nula_vezes += 1

df = pd.DataFrame({
    "Teste T-Student": [tstudent_vezes],
    "Teste Mann-Whitney-U": [num_execucoes - tstudent_vezes]
}, index=["Frequência"])

print(str(df)+"\n\n")

df = pd.DataFrame({
    "Hipótese nula rejeitada": [rejeitar_nula_vezes],
    "Hipótese nula não rejeitada": [num_execucoes - rejeitar_nula_vezes]
}, index=["Frequência"])

print(df)

            Teste T-Student  Teste Mann-Whitney-U
Frequência               33                   967


            Hipótese nula rejeitada  Hipótese nula não rejeitada
Frequência                       60                          940
